In [1]:
import requests
import json
import numpy as np
import pandas as pd
import time
import datetime
import jdatetime

In [2]:
stores_df = pd.read_excel('stores_df_final.xlsx')

In [3]:
stores_df = stores_df.drop('Unnamed: 0',axis=1)

In [4]:
stores_df = stores_df.fillna('---')

In [5]:
stores_list = stores_df.to_dict('records')

In [6]:
def extract_type(x):
    
    if('پنیر' in x.split(' ')):
        return 'پنیر'
    elif('شیر' in x.split(' ')):
        return 'شیر'
    elif('ماست' in x.split(' ')):
        return 'ماست'
    elif('دوغ' in x.split(' ')):
        return 'دوغ'
    elif('کره' in x.split(' ')):
        return 'کره'
    elif('خامه' in x.split(' ')):
        return 'خامه'
    elif('کشک' in x.split(' ')):
        return 'کشک'
    elif('بستنی' in x.split(' ')):
        return 'بستنی'
    else:
        return 'سایر'

In [7]:
products = []

dairy_id = 278345

for store in stores_list:

    store_code = store['store_code']

    products_base_url = 'https://core.snapp.market/api/v2/vendors/{}/products?categories%5B%5D={}'.format(store_code,dairy_id)

    number_of_products = '---'

    j=1
    
    while ((j<5) and (number_of_products=='---')):
        try:
            number_of_products = json.loads(requests.get(products_base_url,timeout=5).text)['metadata']['pagination']['total']
        except:
            j=j+1

    limit = 50

    if(number_of_products!='---'):
    
        for off in range(0,number_of_products,limit):

            products_url = 'https://core.snapp.market/api/v2/vendors/{}/products?categories%5B%5D={}&limit={}&offset={}'.format(store_code,dairy_id,limit,off)

            products_list = '---'

            k=1
            
            while ((k<5) and (products_list=='---')):
                try:
                    products_list = json.loads(requests.get(products_url,timeout=5).text)['results']
                except:
                    k=k+1
                    
            if(products_list!='---'):

                for product in products_list:

                    product_dict = {}

                    try:
                        product_dict['product_id'] = product['id']
                    except:
                        product_dict['product_id'] = '---'
                    try:
                        product_dict['product_name'] = product['title']
                    except:
                        product_dict['product_name'] = '---'
                    try:
                        product_dict['product_discount_percent'] = product['discount_percent']
                    except:
                        product_dict['product_discount_percent'] = '---'
                    try:
                        product_dict['product_main_price'] = product['price']
                    except:
                        product_dict['product_main_price'] = '---'
                    try:
                        product_dict['product_selling_price'] = product['discounted_price']
                    except:
                        product_dict['product_selling_price'] = '---'
                    try:
                        product_dict['product_brand_name'] = product['brand']['title']
                    except:
                        product_dict['product_brand_name'] = '---'
                    try:
                        product_dict['product_category'] = extract_type(product['title'])
                    except:
                        product_dict['product_category'] = '---'
                    try:
                        product_dict['product_max_order'] = product['max_order_cap']
                    except:
                        product_dict['product_max_order'] = '---'
                    try:
                        product_dict['product_store_name'] = store['store_title']
                    except:
                        product_dict['product_store_name'] = '---'
                    try:
                        product_dict['product_store_code'] = store['store_code']
                    except:
                        product_dict['product_store_code'] = '---'

                    products.append(product_dict)

     


In [8]:
products_df = pd.DataFrame(products)

In [9]:
products_df['date'] = str(jdatetime.date.today())

In [10]:
products_df['time'] = datetime.datetime.now().strftime("%X")

In [11]:
products_df.to_excel('products_df.xlsx')

In [12]:
# products_df['product_store_code'].value_counts().index

In [13]:
code_id = stores_df[['store_code','store_id']]

In [14]:
products_df.rename(columns = {'product_store_code':'store_code'}, inplace = True)

In [15]:
products_merge = pd.merge(products_df,code_id,on='store_code')

In [16]:
products_merge

,product_id,product_name,product_discount_percent,product_main_price,product_selling_price,product_brand_name,product_category,product_max_order,product_store_name,store_code,date,time,store_id
0,3617448,کره حیوانی ماردینی ۱۰۰ گرمی,25,25000,18750,ماردینی,کره,8,هایپراستار باکری ۲,097w2n,1402-03-01,22:21:25,57350
1,3635245,شیر پرچرب مدت دار هراز ۲۰۰ میلی لیتری,19,8500,6885,هراز,شیر,12,هایپراستار باکری ۲,097w2n,1402-03-01,22:21:25,57350
2,3622353,دوغ بدون گاز گرمادیده کاله ۹۰۰ میلی لیتری,25,11000,8250,کاله,دوغ,8,هایپراستار باکری ۲,097w2n,1402-03-01,22:21:25,57350
3,3660554,پنیر تاپینگ پیتزا پرچرب رنده شده کیمبال ۱ کیلو...,33,135300,90651,کیمبال,پنیر,6,هایپراستار باکری ۲,097w2n,1402-03-01,22:21:25,57350
4,3674855,ماست پرچرب بدون لاکتوز پنج ستاره کاله ۹۰۰ گرمی,21,38000,30020,کاله,ماست,6,هایپراستار باکری ۲,097w2n,1402-03-01,22:21:25,57350
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4203,3675138,پنیر پیتزا موزارلا پرچرب رنده شده دوماس ۲ کیلو...,0,480000,480000,دوماس,پنیر,6,هایپراستار باکری,pznjk2,1402-03-01,22:21:25,26634
4204,3675288,بسته ۶ عددی روغن کره حیوانی پیور فرش ۴۵۰ گرمی,38,1200000,744000,پیور فرش,کره,1,هایپراستار باکری,pznjk2,1402-03-01,22:21:25,26634
4205,3675375,پنیر خامه ای هراز ۳۰۰ گرمی,25,48900,36675,هراز,پنیر,6,هایپراستار باکری,pznjk2,1402-03-01,22:21:25,26634
4206,3656641,پنیر تاپینگ پیتزا پرچرب رنده شده ۲۰۶ ۱ کیلوگرمی,38,186789,115810,٢٠٦,پنیر,0,هایپراستار باکری,pznjk2,1402-03-01,22:21:25,26634


In [19]:
col_names_product = ['کد محصول','نام محصول','درصد تخفیف محصول','قیمت اصلی محصول','قیمت فروش محصول','برند محصول','گروه محصول','حداکثر سفارش محصول','نام فروشگاه','کد فروشگاه','تاریخ','زمان','آیدی فروشگاه']

col_names_store = ['نوع فروشگاه انگلیسی','نوع فروشگاه','آیدی فروشگاه','کد فروشگاه','نام فروشگاه','آدرس فروشگاه','آدرس خلاصه فروشگاه','شماره تلفن فروشگاه','حداقل قیمت برای ارسال رایگان','حداقل قیمت سبد خرید','هزینه ارسال','هزینه ارسال قبل از تخفیف','حداکثر تخفیف فروشگاه','شهر','وضعیت فروشگاه']

In [21]:
with pd.ExcelWriter("Snapmarket_Tehran_" + str(jdatetime.date.today()).replace('-','_') + ".xlsx") as writer:
    products_merge.columns = col_names_product
    products_merge.to_excel(writer, sheet_name="Productss", index=False)
#     stores_df.columns = col_names_store    
#     stores_df.to_excel(writer, sheet_name='Storess', index=False)